# Create imcrop and dlcrop from h&e image (imsrc), roi image (roi mask 1-12), and

In [2]:
OPENSLIDE_PATH = r'C:\Users\Kevin\Desktop\openslide-win64-20221217\bin'

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [3]:
# import pandas as pd
import numpy as np
import os
from PIL import Image
import os
import cv2
Image.MAX_IMAGE_PIXELS = None
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import openslide
from PIL import Image
from matplotlib import pyplot as plt
import skimage.measure
import matplotlib.pyplot as plt
import scipy.stats as stats
import torchvision.transforms as transforms
from glob import glob
from time import time
from skimage.measure import label
import math

In [80]:
# generate imcrop and dlcrop. just simply resize roi image from roisrc in to the size of imcrop and dlcrop (imcrop and dlcrop have same size, raise error if not in the code), and then make the resized roi image binary (it's from 0-12 right now), and simply multiply binary image to im and dl to create imcrop and dlcrop!
def generate_imcrop_dlcrop(imsrc, dlsrc, roisrc, imcrop_save_pth, dlcrop_save_pth):
    roi_image_list = [x for x in os.listdir(roisrc) if x.endswith(".png")]
    dl_image_list = [x for x in os.listdir(dlsrc) if x.endswith(".tif")]
    im_image_list = [x for x in os.listdir(imsrc) if x.endswith(".tif")]
    roi_complete_list = [os.path.join(roisrc,x) for x in roi_image_list]
    dl_complete_list = [os.path.join(dlsrc,x) for x in dl_image_list]
    im_complete_list = [os.path.join(imsrc,x) for x in im_image_list]
    roi_unique_list = [x[:-6] for x in roi_image_list]
    roi_unique_list1 = list(np.unique(roi_unique_list))

    try:
        len(dl_image_list) == len(im_image_list) == len(roi_unique_list1)
    except ValueError:
        print("ROI image, DL image, and H&E image must have the same number of images.")
    for idx in range(0,len(dl_image_list)): #first iterate over 216
        dl_image = Image.open(dl_complete_list[idx])
        im_image = Image.open(im_complete_list[idx])
        dl_array = np.array(dl_image)
        im_array = np.array(im_image)
        print("dl file path",dl_complete_list[idx])
        if not dl_array.shape == (im_array.shape[0],im_array.shape[1]): # height and width must be same
            raise Exception("Size of the H&E and DL image must be the same with each other.")
        repeats = sum(f in roi_unique_list1[idx] for f in roi_unique_list)
        print("repeat rois",repeats)
        for id in range(0,repeats): #iterate over repeats (rois) in each 216 unique images
            roi_image= Image.open(roi_complete_list[idx + id])
            print("roi file path",roi_complete_list[idx + id])

            roi_array = np.array(roi_image)
            print("roi shape before resize",roi_array.shape)
            resized_roi = cv2.resize(roi_array,(dl_array.shape[1],dl_array.shape[0]),interpolation = cv2.INTER_AREA)
            if not resized_roi.shape == dl_array.shape == (im_array.shape[0],im_array.shape[1]):
                print("im shape", im_array.shape)
                print("dl shape", dl_array.shape)
                print("roi shape after resize",resized_roi.shape)
                raise Exception("All three images aren't of same size after resizing, please check.")

            binary_resized_roi = resized_roi > 0 # binary roi
            binary_resized_roi_rgb = np.repeat(binary_resized_roi[:,:,np.newaxis],3,axis=2) #match binary roi to rgb channel
            imcrop = np.multiply(im_array,binary_resized_roi_rgb)
            dlcrop = np.multiply(dl_array,binary_resized_roi)
            save_name = roi_image_list[idx + id].replace(".png","")
            imcrop_save_path = os.path.join(imcrop_save_pth, save_name + ".png")
            dlcrop_save_path = os.path.join(dlcrop_save_pth, save_name + ".png")
            Image.fromarray(imcrop).save(imcrop_save_path)
            Image.fromarray(dlcrop).save(dlcrop_save_path)
    return print("Saved successfully!")

In [ ]:
generate_imcrop_dlcrop(imsrc = r'\\shelter\Kyu\skin_aging\clue_cohort\HE1um', dlsrc = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um',roisrc = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeled_mask_per_roi',imcrop_save_pth = r'\\shelter\Kyu\skin_aging\clue_cohort\imcrop1um', dlcrop_save_pth= r'\\shelter\Kyu\skin_aging\clue_cohort\dlcrop1um')

dl file path \\shelter\Kyu\skin_aging\clue_cohort\DLmask1um\2022-06-07 13.18.40.tif
repeat rois 1
roi file path \\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeled_mask_per_roi\2022-06-07 13.18.40_1.png
roi shape before resize (21140, 21196)
dl file path \\shelter\Kyu\skin_aging\clue_cohort\DLmask1um\2022-06-07 13.29.45.tif
repeat rois 4
roi file path \\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeled_mask_per_roi\2022-06-07 13.29.45_1.png
roi shape before resize (25492, 44088)
roi file path \\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeled_mask_per_roi\2022-06-07 13.29.45_2.png
roi shape before resize (25492, 44088)
roi file path \\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeled_mask_per_roi\2022-06-07 13.29.45_3.png
roi shape before resize (25492, 44088)
roi file path \\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeled_mask_per_roi\2022-06-07 13.29.45_4.png
roi shape before resize (25492, 44088)
dl file path \\shelter\Kyu\skin_aging\c

In [62]:
# roisrc = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeled_mask_per_roi'
# roi_image_list = [x for x in os.listdir(roisrc) if x.endswith(".png")]
# roi_unique_list = [x[:-6] for x in roi_image_list]
# roi_unique_list1 = list(np.unique(roi_unique_list))
# repeats = sum(f in roi_unique_list1[11] for f in roi_unique_list)
# repeats

2